# requirements 

In [1]:
import requests
from bs4 import BeautifulSoup
from PIL import Image 
import os 
import PIL 
import glob
import re
import pandas as pd

# Scrapping

In [4]:
#website to scrap
url = 'http://moto-occasion.motorevue.com/motos?page=2'
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")
source = 'moto-occasion'

In [9]:
#import previously scrapped 
df_import = pd.read_csv(f'{source}.csv')

#initialize list for dataframe
source_ls=[]
url_bike_ls=[]
price_ls=[]
vendor_type_ls=[]
vendor_name_ls=[]
vendor_city_ls=[]
vendor_country_ls=[]
reference_ls=[]
uniq_id_ls=[]
bike_type_ls=[]
bike_size_ls=[]
bike_year_ls=[]
bike_km_ls=[]
bike_brand_ls=[]
bike_model_ls=[]
bike_description_ls=[]

for bike in soup.find_all("li", class_="list-items__item"):
    #find the url for one bike
    link = bike.find("a", class_="list-items__link").get('href')
    url_site = 'http://moto-occasion.motorevue.com'
    #generate the url for one bike
    url_bike = url_site + link 
    response_bike = requests.get(url_bike)
    
    #one bike soup
    bike_soup = BeautifulSoup(response_bike.content, "html.parser")
    
    #vendor contact soup & info
    vendor_info = bike_soup.find("div", class_="item-contact__content").find_all("div", class_="grid-5-8 phab-7-10")
    
    ##unique reference
    reference = vendor_info[2].text.strip()
    uniq_id = "-".join([source, reference])
    
    if uniq_id not in df_import['uniq_id'].tolist():
    
        vendor_type = vendor_info[0].text.strip().upper()
        vendor_name = vendor_info[1].text.strip().replace('\t', '').replace('\n', ', ')
        vendor_localisation = vendor_info[3].text.strip().replace('\t', '').split('(')
        vendor_city = vendor_localisation[0]
        vendor_country = vendor_localisation[1].replace(")","").upper()

        #moto description
        bike_info_1 = bike_soup.find("div", class_="item-description").find("div", class_="grids").find_all("span", class_="item-data")
        bike_type = bike_info_1[0].text.strip().upper()
        bike_size = int(bike_info_1[1].text.replace("cm3","").strip())
        bike_year = int(bike_info_1[2].text.strip())
        bike_km = int(bike_info_1[3].text.strip().replace("km","").strip())

        bike_info_2 = bike_soup.find("div", class_="item-description").find("div", class_="grids").find_all("h2", class_="item-data")
        bike_brand = bike_info_2[0].text.strip()
        bike_model = bike_info_2[1].text.strip()

        bike_description = bike_soup.find(id="infos").text.strip()

        #price
        price_soup = bike_soup.find("p", class_="item__price").text
        price = price_soup.replace("€", "").replace("TTC", "").replace(" ","")
        price=float("".join(price.split()).replace(",","."))

        #pictures
        img_soup = bike_soup.find("div", class_="slideshow__container").find_all("img")
        k=0
        for image in img_soup:
            image_url = url_site + image['src']
            img_data = requests.get(image_url).content
            with open(f'img/{uniq_id}-{k}.jpg', 'wb') as handler:
                handler.write(img_data)

            image = Image.open(f'img/{uniq_id}-{k}.jpg') 
            ratio = image.size[0] / image.size[1]
            image = image.resize((300,int(300/ratio)))
            image.save(f'img/{uniq_id}-{k}.jpg',optimize = True, quality = 50)
            k+=1

        url_bike_ls.append(url_bike)
        price_ls.append(price)
        vendor_type_ls.append(vendor_type)
        vendor_name_ls.append(vendor_name)
        vendor_city_ls.append(vendor_city)
        vendor_country_ls.append(vendor_country)
        reference_ls.append(reference)
        uniq_id_ls.append(uniq_id)
        bike_type_ls.append(bike_type)
        bike_size_ls.append(bike_size)
        bike_year_ls.append(bike_year)
        bike_km_ls.append(bike_km)
        bike_brand_ls.append(bike_brand)
        bike_model_ls.append(bike_model)
        bike_description_ls.append(bike_description)
    else:
        print('skip')

    df = pd.DataFrame(zip(
        uniq_id_ls,
        reference_ls,
        url_bike_ls,
        bike_type_ls,
        bike_size_ls,
        bike_year_ls,
        bike_km_ls,
        bike_brand_ls,
        bike_model_ls,
        bike_description_ls,
        price_ls,
        vendor_type_ls,
        vendor_name_ls,
        vendor_city_ls,
        vendor_country_ls),
                     columns=[
                        'uniq_id',
                        'reference',
                        'url_bike',
                        'bike_type',
                        'bike_size',
                        'bike_year',
                        'bike_km',
                        'bike_brand',
                        'bike_model',
                        'bike_description',
                        'price',
                        'vendor_type',
                        'vendor_name',
                        'vendor_city',
                        'vendor_country'])

df['source'] = source

# concatenating old and new
final_df = pd.concat([df_import, df], axis=0)

# export to csv
final_df.to_csv(f'{source}.csv', index=False)

#final_df.bike_model.value_counts()

skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip


X-MAX 125                3
MT-125                   2
BANDIT GSF 650 N         2
1290 SUPER DUKE GT       1
SV N 650                 1
MP3 500 LT               1
XJ6 N 600                1
CB 500                   1
CB X 500                 1
Speed TRIPLE 1050 S/R    1
R 1200                   1
R 1200 GS                1
S 1000 XR                1
ZX-6R 636                1
MP3 LT 300               1
Beverly i.e. 300         1
CB1000R                  1
PCX 125                  1
MT-07 700                1
Liberty                  1
Super Adventure 1290     1
CBF 125                  1
SH 125                   1
Z1000                    1
Nmax                     1
Goldwing 1800            1
Name: bike_model, dtype: int64